**Scraping the Zillow website to get additional information for the IA State Housing Dataset.**

In [ ]:
!pip install BeautifulSoup4
!pip install unicodecsv

**Imporsting required libraries**

In [ ]:
pip install requests-html

In [ ]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd 'gdrive/MyDrive'

[Errno 2] No such file or directory: 'gdrive/MyDrive'
/content/gdrive/MyDrive


**IA housing dataset**


In [ ]:
df = pd.read_csv('IA_Dataset_Realestate.csv')

In [ ]:
df.head()

,rank,property_id,address,latitude,longitude,price,currency,bathrooms,bedrooms,area,land_area,zestimate,rent_zestimate,days_on_zillow,sold_date,is_zillow_owned,image,listing_type,broker_name,input,property_url,listing_url
0,532,247261885,"2863 Spring Rose Cir, Coralville, IA 52241",41.704755,-91.613700,529900.0,USD,4.0,4.0,2891.0 sqft,NaN,NaN,1032.0,2.0,NaN,False,https://photos.zillowstatic.com/fp/eb14d416f9d...,House for sale,NaN,https://www.zillow.com/homes/IA_rb/,https://www.zillow.com/homedetails/2863-Spring...,https://www.zillow.com/homes/IA_rb/14_p/
1,51,76679888,"215 3rd Ave SW, Waverly, IA 50677",42.722563,-92.473925,164950.0,USD,2.0,3.0,1711.0 sqft,NaN,143400.0,1200.0,0.0,NaN,False,https://photos.zillowstatic.com/fp/8d31246462f...,House for sale,Coldwell Banker Elevated Real Estate,https://www.zillow.com/homes/IA_rb/,https://www.zillow.com/homedetails/215-3rd-Ave...,https://www.zillow.com/homes/IA_rb/2_p/
2,231,238337348,"813 Cherry St NW, Bondurant, IA 50035",41.710072,-93.466078,234900.0,USD,2.0,3.0,1168.0 sqft,NaN,NaN,688.0,0.0,NaN,False,https://photos.zillowstatic.com/fp/178a6eebb6d...,New construction,RE/MAX Concepts,https://www.zillow.com/homes/IA_rb/,https://www.zillow.com/homedetails/813-Cherry-...,https://www.zillow.com/homes/IA_rb/6_p/
3,796,803919,"2405 36th St, Des Moines, IA 50310",41.616056,-93.665458,99900.0,USD,1.0,2.0,752.0 sqft,NaN,122000.0,1116.0,3.0,NaN,False,https://photos.zillowstatic.com/fp/0c099b048cb...,House for sale,Keller Williams Realty GDM,https://www.zillow.com/homes/IA_rb/,https://www.zillow.com/homedetails/2405-36th-S...,https://www.zillow.com/homes/IA_rb/20_p/
4,429,76832920,"1418 Dorene Blvd, Carter Lake, IA 51510",41.288925,-95.910227,117500.0,USD,1.0,2.0,925.0 sqft,NaN,116400.0,1132.0,1.0,NaN,False,https://photos.zillowstatic.com/fp/5926ffffd68...,House for sale,NaN,https://www.zillow.com/homes/IA_rb/,https://www.zillow.com/homedetails/1418-Dorene...,https://www.zillow.com/homes/IA_rb/11_p/


**Taking the addresses and their equivalent zipcodes**

In [ ]:
df['zip'] = df['address'].str.extract(r'(\d{5}\-?\d{0,4})')

In [ ]:
addr = df['address']
zip = df['zip']
print(addr,zip)

0      2863 Spring Rose Cir, Coralville, IA 52241
1               215 3rd Ave SW, Waverly, IA 50677
2           813 Cherry St NW, Bondurant, IA 50035
3              2405 36th St, Des Moines, IA 50310
4         1418 Dorene Blvd, Carter Lake, IA 51510
                          ...                    
752               229 Quince Rd, Walnut, IA 51577
753           1710 Forest Ave, Waterloo, IA 50702
754         1222 Drake Ave, Centerville, IA 52544
755             205 1st Ave S, Humboldt, IA 50548
756        417 S Marion Ave, Washington, IA 52353
Name: address, Length: 757, dtype: object 0      52241
1      50677
2      50035
3      50310
4      51510
       ...  
752    51577
753    50702
754    52544
755    50548
756    52353
Name: zip, Length: 757, dtype: object


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

**Modifying the address value to URL equivalent value**


In [ ]:
for index, add in enumerate(addr):
  new = add.replace(' ','-')
  addr[index] = new

print(addr)
print(zip)

0      2863-Spring-Rose-Cir,-Coralville,-IA-52241
1               215-3rd-Ave-SW,-Waverly,-IA-50677
2           813-Cherry-St-NW,-Bondurant,-IA-50035
3              2405-36th-St,-Des-Moines,-IA-50310
4         1418-Dorene-Blvd,-Carter-Lake,-IA-51510
                          ...                    
752               229-Quince-Rd,-Walnut,-IA-51577
753           1710-Forest-Ave,-Waterloo,-IA-50702
754         1222-Drake-Ave,-Centerville,-IA-52544
755             205-1st-Ave-S,-Humboldt,-IA-50548
756        417-S-Marion-Ave,-Washington,-IA-52353
Name: address, Length: 757, dtype: object
0      52241
1      50677
2      50035
3      50310
4      51510
       ...  
752    51577
753    50702
754    52544
755    50548
756    52353
Name: zip, Length: 757, dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


**Now we create the url, fetch the URL with proper headers, so that we can bypass the CAPTCHA and then scrap the required data from the site**

In [ ]:

import string
t = str.maketrans('','',string.ascii_letters)

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}


properties_list = []
title_list = ['BuildingType', 'CalendarYear built', 'HeatingHeating', 'SnowflakeCooling', 'ParkingParking', 'HOAHOA', 'LotLot', 'Price Square FeetPrice/sqft']

for index, add in enumerate(addr):
  with requests.Session() as req_session:

  
    current_url = 'https://www.zillow.com/homes/'+add+'_rb/'
    print(current_url)
  
    resp = req_session.get(current_url, headers=headers)
    soup = BeautifulSoup(resp.content, 'html.parser')
 
    walk_current_url = 'https://www.walkscore.com/score/'+str.lower(add)
    print(walk_current_url)
   
    walk_resp = req_session.get(walk_current_url, headers=headers)
    walk_soup = BeautifulSoup(walk_resp.content, 'html.parser')

  
    school_current_url = 'https://www.niche.com/places-to-live/z/'+ zip[index]
    print(school_current_url)

  
    school_resp = req_session.get(school_current_url, headers=headers)
    school_soup = BeautifulSoup(school_resp.content, 'html.parser')
    

    properties = {}

    properties['address'] = add.replace('-',' ')
    properties['zip'] = zip[index]
    
   
    a_address = properties['address'].split(', ')
    city = a_address[-2]
    crime_current_url = 'https://247wallst.com/city/crime-in-' + str.lower(city).replace(' ','-') + '-iowa'
    
    crime_resp = req_session.get(crime_current_url, headers=headers)
    crime_soup = BeautifulSoup(crime_resp.content, 'html.parser')
    print(crime_current_url)

    for i,data in enumerate(soup.find_all('li', class_='ds-home-fact-list-item')):
      raw_data = data.getText().split(':')
      raw_data[1] = raw_data[1].replace('$','')
      raw_data[1] = raw_data[1].replace(',','')
      print(raw_data)
      if(raw_data[0] == title_list[0]):                   
        if(raw_data[1] and raw_data[1] != 'No Data'):
          properties[title_list[0]] = raw_data[1]
        else:
          properties[title_list[0]] = None
      elif(raw_data[0] == title_list[1]):                 
        if(raw_data[1] and raw_data[1] != 'No Data'):
          properties[title_list[1]] = raw_data[1]
        else:
          properties[title_list[1]] = None
      elif(raw_data[0] == title_list[2]):                 
        if(raw_data[1] and raw_data[1] != 'No Data'):
          properties[title_list[2]] = raw_data[1]
        else:
          properties[title_list[2]] = None
      elif(raw_data[0] == title_list[3]):                 
        if(raw_data[1] and raw_data[1] != 'No Data'):
          properties[title_list[3]] = raw_data[1]
        else:
          properties[title_list[3]] = None
      elif(raw_data[0] == title_list[4]):                 
        if(raw_data[1] and raw_data[1] != 'No Data'):
          value = raw_data[1].split(' ')[0]
          value = value.translate(t)
          if value == '-':
            value = None
          properties[title_list[4]] = value
        else:
          properties[title_list[4]] = None
      elif(raw_data[0] == title_list[5]):                
        if(raw_data[1] and raw_data[1] != 'No Data'):
          raw_data[1] = raw_data[1].replace('$','')
          raw_data[1] = raw_data[1].replace(',','')
          raw_data[1] = raw_data[1].translate(t)
          value = raw_data[1].split('/')[0]
          value = value.replace(',','')
          properties[title_list[5]] = round(float(value))
        else:
          properties[title_list[5]] = None
      elif(raw_data[0] == title_list[6]):                 
        if(raw_data[1] and raw_data[1] != 'No Data'):
          if 'acre' in raw_data[1].lower():
            raw_data[1] = raw_data[1].translate(t)
            raw_data[1] = str(round(float(raw_data[1])*43560))
          raw_data[1] = raw_data[1].translate(t)
          value = raw_data[1].split(' ')[0]
          value = value.replace(',','')
          properties[title_list[6]] = round(float(value))
        else:
          properties[title_list[6]] = None
      elif(raw_data[0] == title_list[7]):                 
        if(raw_data[1] and raw_data[1] != 'No Data'):
          raw_data[1] = raw_data[1].translate(t)
          value = raw_data[1].split(' ')[0]
          value = value.replace(',','')
          properties[title_list[7]] = round(float(value))
        else:
          properties[title_list[7]] = None
    

    def has_numbers(inputString):
      return any(char.isdigit() for char in inputString)

   
    for i,data in enumerate(soup.find_all('span', class_='Text-c11n-8-48-0__sc-aiai24-0 sc-pZcYF kOhnmm')):
      print(data.getText())
     
      raw_data = data.getText()
      raw_data = raw_data.replace('$','')
      raw_data = raw_data.replace(',','')
      raw_data = raw_data.split('/')[0]
      
    
      if(i == 0):
      
        print('price: ' + raw_data)
        if has_numbers(raw_data):
          price = int(raw_data)
          loan_amount = price*0.85
          mortgage_yearly = loan_amount*0.03
          mortgage_monthly = mortgage_yearly/12
          properties['Mortgage fees per month'] = round(mortgage_monthly)
          properties['Mortgage fees per year'] = round(mortgage_yearly)

          print(mortgage_monthly)
          print(mortgage_yearly)
      
   

    # Get Walk Score
    for i,data in enumerate(walk_soup.find_all('div',class_='block-header-badge score-info-link')):
      for img in data.find_all('img', alt=True):
        alt_text = img['alt']
        if "walk" in str.lower(alt_text):
          walk_score = alt_text.split(" ")[0]
          print('walk score: '+ walk_score)
          properties['Walk Score'] = walk_score
        else:
          walk_score = None
    # Get school ranking 
    school_rank = 0
    for i,data in enumerate(school_soup.find_all('span',class_='rankings__collection__ordinal')):
      raw_data = data.getText()
      print('school rank: ' + raw_data)
      school_rank = raw_data
      properties['School Rank'] = int(school_rank)

    # Get Crime rate
    for i,data in enumerate(crime_soup.find_all('div',id='violentCrime')):
      raw_data = data.getText()
      inx = raw_data.index('violent crimes')
      substr = raw_data[inx-20:inx]
      violent_crime = substr.split(' ')[-1]
      violent_crime = property_crime.replace(',','')
      print('violent crime rate: ' + property_crime)
      properties['Violent Crime'] = int(violent_crime)

    for i,data in enumerate(crime_soup.find_all('div',id='propertyCrime')):
      raw_data = data.getText()
      inx = raw_data.index(' for every')
      substr = raw_data[inx-20:inx]
      property_crime = substr.split(' ')[-1]
      property_crime = property_crime.replace(',','')
      print('property crime rate: ' + property_crime)
      properties['Property Crime'] = int(property_crime)
    
   
    for i,data in enumerate(soup.find_all('div',class_='Text-c11n-8-48-0__sc-aiai24-0 hdp__sc-19crqy3-1 kbUEOB hdp__sc-19crqy3-7 fMlHYK')):
     
      raw_data = data;
      print(raw_data)
      if("over the past 12 months." in raw_data):         
        raw_data = raw_data.replace(".","")[raw_data.index('%')-5:].split('%')[0].strip()
        properties['price_rise_rate'] = raw_data

      if("Zillow predicts the home values" in raw_data):  
        try:
          val = (float)(raw_data.replace("in the next year.","")[raw_data.index('%')-4:].split('%')[0].strip())
        except:
          val = (float)(raw_data.replace("in the next year.","")[raw_data.index('%')-2:].split('%')[0].strip())
        properties['price_rise_predicted'] = val

      if("than the median home" in raw_data):             
        try:
          val = (float)(raw_data[raw_data.index('%')-4:].split(' ')[0].replace('%','').strip())
        except: 
          val = (float)(raw_data[raw_data.index('%')-1:].split(' ')[0].replace('%','').strip())
        properties['med_neghbhood_price_rate'] = val

      if("median Zestimate" in raw_data):                 
        raw_data = raw_data.replace("The median Zestimate® for this neighborhood is $","").strip().replace(",","").replace(".","")
        properties['MedianZestimate'] = raw_data

    properties_list.append(properties)                     
    
print(properties_list)


import unicodecsv as csv
with open("final_scrapped_properties.csv",'wb')as csvfile:
		fieldnames = ['address','zip','BuildingType', 'CalendarYear built', 'HeatingHeating', 'SnowflakeCooling', 'ParkingParking', 'HOAHOA', 'LotLot', 'Price Square FeetPrice/sqft', 'Rent', 'Mortgage fees per month','Mortgage fees per year', 'Walk Score', 'School Rank','Violent Crime','Property Crime']
		writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
		writer.writeheader()
		for row in properties_list:
			writer.writerow(row)

Streaming output truncated to the last 5000 lines.
478.125
5737.5
$225,000
walk score: 4
school rank: 36
violent crime rate: 1413
property crime rate: 4120
https://www.zillow.com/homes/4406-Ross-Rd,-Ames,-IA-50014_rb/
https://www.walkscore.com/score/4406-ross-rd,-ames,-ia-50014
https://www.niche.com/places-to-live/z/50014
https://247wallst.com/city/crime-in-ames-iowa
['BuildingType', 'Single Family Residence']
['CalendarYear built', '1966']
['HeatingHeating', 'Forced Air Gas Natural Gas']
['SnowflakeCooling', 'No Data']
['ParkingParking', '1 Garage space']
['LotLot', '8276 sqft']
['Price Square FeetPrice/sqft', '240']
$219,000
price: 219000
465.375
5584.5
$219,000
walk score: Walk
school rank: 9
school rank: 26
school rank: 52
violent crime rate: 4120
property crime rate: 1716
https://www.zillow.com/homes/619-19th-Ave-N,-Clinton,-IA-52732_rb/
https://www.walkscore.com/score/619-19th-ave-n,-clinton,-ia-52732
https://www.niche.com/places-to-live/z/52732
https://247wallst.com/city/crime-i